In [1]:
import numpy as np
import optuna
import os
import pandas as pd
import time

working_dir = os.getcwd()
print(working_dir)


/home/rachtrx/mambaforge/envs/sc1015/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/mnt/c/Users/rachm/OneDrive/projects/personal_projects/sc2001/project_1


In [2]:
ARR_LENS = list(range(10**3, 10**7 + 1, 10**5))
S_THRESHOLDS = np.linspace(0.01, 1.0, num=20)

In [3]:
class DataLoader:
    def __init__(self, arr_len):
        self.arr_len = arr_len
        
    def generate_data(self):
        return np.random.randint(1, 100, self.arr_len)

In [4]:
class Model:

    def __init__(self, array, S):
        self.S = S   
        self.array = array

    def train(self):
        self.execution_time = None
        self.key_cmp = 0
        start_time = time.perf_counter()
        self.merge_insertion_sort(0, len(self.array)-1)
        end_time = time.perf_counter()
        self.execution_time = end_time - start_time

    def merge(self, n, m):
        mid = (n+m)//2
        a=n
        b=mid+1
        i=None
        tmp=None

        if m-n<=0:
            return
        while a<=mid and b<=m:
            self.key_cmp += 1
            cmp = self.array[a] - self.array[b]
            if cmp>0:
                tmp = self.array[b]
                b += 1
                mid += 1
                i = mid
                while i>a:
                    self.array[i] = self.array[i-1]
                    i -= 1
                self.array[a] = tmp
                a += 1
            elif cmp<0:
                a += 1
            else:
                if a==mid and b==m:
                    break
                tmp = self.array[b]
                a += 1
                b += 1
                mid += 1
                i = mid
                while i>a:
                    self.array[i] = self.array[i-1]
                    i -= 1
                self.array[a] = tmp
                a += 1

    def merge_insertion_sort(self, n, m):
        
        if (m-n <= 0):
            return
        
        if m-n+1 <= self.S:
            self.insertion_sort(n, m)
            return
        
        mid = (n+m)//2
        if (m-n>1):
            self.merge_insertion_sort(n, mid)
            self.merge_insertion_sort(mid+1, m)
        
        self.merge(n, m)

    def merge_sort(self, n, m):
        if (m-n <= 0):
            return
        
        mid = (n+m)//2
        if (m-n>1):
            self.merge_sort(n, mid)
            self.merge_sort(mid+1, m)
        
        self.merge(n, m)

    def insertion_sort(self, n, m):
        i = n+1
        while i <= m:
            j = i
            while j > n:
                self.key_cmp += 1
                if self.array[j] < self.array[j-1]:
                    tmp = self.array[j]
                    self.array[j] = self.array[j-1]
                    self.array[j-1] = tmp
                    j -= 1
                else:
                    break
            i+=1

In [ ]:
results = []

for arr_len in ARR_LENS:
    print(f"Running for arrays of length {arr_len}")
    dataloader = DataLoader(arr_len=arr_len)
    array = dataloader.generate_data()
    
    for S_ratio in S_THRESHOLDS:
        S = max(1, int(S_ratio * arr_len))
        
        model = Model(array, S)
        model.train()
        
        results.append({
            "Array Length": arr_len,
            "Threshold (S)": S,
            "Threshold Ratio": S_ratio,
            "Execution Time": model.execution_time,
            "Key Comparisons": model.key_cmp
        })

df_results = pd.DataFrame(results)

df_results.to_parquet("sorting_experiment_results.parquet", index=False)

Running for arrays of length 1000
Running with threshold ratio of 0.01
Running with threshold ratio of 0.06210526315789474
Running with threshold ratio of 0.11421052631578947
Running with threshold ratio of 0.16631578947368422
Running with threshold ratio of 0.21842105263157896
Running with threshold ratio of 0.2705263157894737
Running with threshold ratio of 0.32263157894736844
Running with threshold ratio of 0.37473684210526315
Running with threshold ratio of 0.4268421052631579
Running with threshold ratio of 0.4789473684210527
Running with threshold ratio of 0.5310526315789474
Running with threshold ratio of 0.5831578947368421
Running with threshold ratio of 0.6352631578947369
Running with threshold ratio of 0.6873684210526316
Running with threshold ratio of 0.7394736842105263
Running with threshold ratio of 0.791578947368421
Running with threshold ratio of 0.8436842105263158
Running with threshold ratio of 0.8957894736842106
Running with threshold ratio of 0.9478947368421053
Runnin

KeyboardInterrupt: 